# Challenge - Clickbait Title Detection

# Background information

Clickbait titles and thumbnails are plagueing the internet and lead to lesser user satisfaction with services like YouTube or news servers. Due to the amount on new content on these sites, it is impossible to control content manually. That is why giants like Facebook(Meta), Twitter, Amazon or Google(Alphabet) are investing huge resources towards creating NLP systems that ae able to curate internet enviroment autonomously.

To make our Clickbait Detection model we will use Bag of Words encoding and sequential model.

# Data

We will use clickbait data, which you can download from our GitHub.It has 2 categories ("headline" - containing titles & clickbait - containing the labels). As the separator, we use ";" because comma can be problematic on some system due to commas being also used in the text.

In [1]:
# Importing required libraries and download NLTK resources
import numpy as np
from numpy import array
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [3]:
# Load data into dataframe
DATA_PATH = 'clickbait_data.csv'
df = pd.read_csv(DATA_PATH)
df.dropna(subset = ["clickbait"], inplace=True)
np.random.shuffle(df.values)

# Load corpus
corpus = list(df["headline"])
labels = array(df["clickbait"])

# Preprocessing

In NLP there are multiple ways how to approach preprocessing. It is more or less up to us, what kinds of preprocessing we want to do and not always are all of them helpful.
The most common preprocessing techniques are:
- Removing stopwords
- Lemmatizaton
- Stemming

In [5]:
# Get all unique words
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        if word not in stopwords:
            word = stemmer.stem(word)
            word = lemmatizer.lemmatize(word)
            all_words.append(word)
unique_words = set(all_words)
print(len(unique_words))

18538


# Embeddings

Creating embeddings could be also seen as a form of preprocessing, which is maybe the most important choice you make when building NLP model. We are using the Bag of Words approach, which is very simplistic. They are better embeddings for this task, but there are situation when BoW is the best option.

In [6]:
# Create embeddings
vocab_length = len(unique_words) + 5
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
padded_sentences = pad_sequences(embedded_sentences, padding='post')

In [14]:
#Split data
from sklearn.model_selection import train_test_split
data_train, data_test, labels_train, labels_test = train_test_split(embedded_sentences, labels, test_size=0.3)

In [15]:
# Create model
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=padded_sentences.shape[1]))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [16]:
#Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [20]:
#Fit model
model.fit(data_train, labels_train, epochs=5, verbose=1)

Epoch 1/5
700/700 [==============================] - 5s 8ms/step - loss: 0.0051 - acc: 0.9996
Epoch 2/5
700/700 [==============================] - 6s 8ms/step - loss: 0.0030 - acc: 0.9998
Epoch 3/5
700/700 [==============================] - 5s 7ms/step - loss: 0.0018 - acc: 0.9999
Epoch 4/5
700/700 [==============================] - 5s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 5/5
700/700 [==============================] - 6s 8ms/step - loss: 6.9430e-04 - acc: 1.0000


In [ ]:
#Count accuracy
loss, accuracy = model.evaluate(data_test, labels_test, verbose=0,batch_size=10)
print('Accuracy: %f' % (accuracy*100))